# Recurrent Neural Networks

Gabs DiLiegro, London Kasper, Carys LeKander

Dataset: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset?select=spam.csv

## Preparation

Our dataset is a collection of text messages that are identified as normal or spam texts.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('spam.csv',encoding='latin-1')
df = df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


The first column of our dataset identifies if it is a spam text. We used label encoding below to change the column to be 0 for a normal text and 1 for a spam text. 

In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

le = preprocessing.LabelEncoder()
le.fit(df['v1'])
df['v1'] = le.transform(df['v1'])

df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Looking at the totals for each category, we can see that there are many more normal text messages than spam messages in our dataset. 

In [3]:
totals = df['v1'].value_counts()
totals

0    4825
1     747
Name: v1, dtype: int64

The longest text message in our dataset is 171 which we will use for our max length during tokenization.


In [4]:
max_words = 0
for i in df.v2:
    if len(i.split()) > max_words:
        max_words = len(i.split())
max_words

171

For our tokenization, we converted each word to an integer using the Keras Tokenizer.

In [5]:
%%time
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

NUM_TOP_WORDS = None # use entire vocabulary!
MAX_ART_LEN = max_words # maximum and minimum number of words

#tokenize the text
tokenizer = Tokenizer(num_words=NUM_TOP_WORDS)
tokenizer.fit_on_texts(df.v2)
# save as sequences with integers replacing words
sequences = tokenizer.texts_to_sequences(df.v2)

word_index = tokenizer.word_index
NUM_TOP_WORDS = len(word_index) if NUM_TOP_WORDS==None else NUM_TOP_WORDS
top_words = min((len(word_index),NUM_TOP_WORDS))
print('Found %s unique tokens. Distilled to %d top words.' % (len(word_index),top_words))

X = pad_sequences(sequences, maxlen=MAX_ART_LEN)
print('Shape of data tensor:', X.shape)
y = df.v1

2022-12-13 16:51:50.879426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 8920 unique tokens. Distilled to 8920 top words.
Shape of data tensor: (5572, 171)
CPU times: user 8.86 s, sys: 1.45 s, total: 10.3 s
Wall time: 20.9 s


For our business case, we want to notify users when they recieve a text that may be spam. If we misclassify something as spam, the user may be slightly annoyed. However, if we do not clasify something as spam when it is, it could lead to the user trusting a fraudulent text. Therefore we want to minimize the number of false negative (number of spam texts classified as normal texts), so we will use recall to measure our algorithm's performance.

We chose to use a Stratified 10-fold cross validation for dividing our data into training and testing. We chose this method because our dataset is smaller (5572) and there are not many examples of spam texts (747). We want to make sure that we represent the entire dataset in equal proportion.

MAYBE ADD MORE EXPLANATION TO THIS

In [6]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X, y)

## Modeling

***Investigate at least two different recurrent network architectures (perhaps LSTM and GRU). Alternatively, you may also choose one recurrent network and one convolutional network. Be sure to use an embedding layer (try to use a pre-trained embedding, if possible). Adjust hyper-parameters of the networks as needed to improve generalization performance (train a total of at least four models). Discuss the performance of each network and compare them.***

In [8]:
%%time
EMBED_SIZE = 100
# the embed size should match the file you load glove from
embeddings_index = {}
f = open('large_data/glove.6b/glove.6B.100d.txt')
# save key/array pairs of the embeddings
#  the key of the dictionary is the word, the array is the embedding
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# now fill in the matrix, using the ordering from the
#  keras word tokenizer from before
found_words = 0
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be ALL-ZEROS
        embedding_matrix[i] = embedding_vector
        found_words = found_words+1

print("Embedding Shape:",embedding_matrix.shape, "\n",
      "Total words found:",found_words, "\n",
      "Percentage:",100*found_words/embedding_matrix.shape[0])

Found 400000 word vectors.
Embedding Shape: (8921, 100) 
 Total words found: 6518 
 Percentage: 73.06355789709674
CPU times: user 23.1 s, sys: 1.68 s, total: 24.8 s
Wall time: 1min 10s


In [11]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, LSTM, GRU
from tensorflow.keras.layers import Embedding

In [12]:
# save this embedding now
embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],# here is the embedding getting saved
                            input_length=MAX_ART_LEN,
                            trainable=False)

In [14]:
lstm = Sequential()
lstm.add(embedding_layer)
lstm.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
lstm.add(Dense(2, activation='sigmoid'))
lstm.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 171, 100)          892100    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 972,702
Trainable params: 80,602
Non-trainable params: 892,100
_________________________________________________________________
None


***Use the method of train/test splitting and evaluation criteria that you argued for at the beginning of the lab. Visualize the results of all the RNNs you trained.  Use proper statistical comparison techniques to determine which method(s) is (are) superior.***